# Core: Applying 3-Category Classifier for Scoring
Apply trained model to samples in catalog in order to assess model performance.  

This notebook addresses only scoring, not mapping.
  
Date: 2019-06-27  
Author: Peter Kerins  

### Import statements
(may be over-inclusive)

In [1]:
# typical, comprehensive imports
import warnings
warnings.filterwarnings('ignore')
#
import os
import sys
import pickle
from pprint import pprint
#
import numpy as np
get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt
import pandas as pd

from importlib import reload

# from tensorflow.keras.models import load_model
# from tensorflow.keras import models
# from tensorflow.keras import layers
# from tensorflow.keras.layers import Dropout
# from tensorflow.keras.utils import to_categorical

import tensorflow as tf

# import tensorflow.keras as keras
# import tensorflow.keras.backend as K
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
# from tensorflow.keras.layers import Conv2D, MaxPooling2D
# from tensorflow.keras.layers import Input, Add, Lambda
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, History

import descarteslabs as dl

ULU_REPO = os.environ["ULU_REPO"]
if not ULU_REPO in sys.path:
    sys.path.append(ULU_REPO)
# ulu_utils = ULU_REPO+'/utils'
# if not ulu_utils in sys.path:
#     sys.path.append(ulu_utils)
print (sys.path)

import utils.util_chips as util_chips
import utils.util_workflow as util_workflow
from utils.catalog_generator import CatalogGenerator
import utils.util_scoring as util_scoring


['/home/Peter.Kerins/anaconda3/envs/geoml/lib/python36.zip', '/home/Peter.Kerins/anaconda3/envs/geoml/lib/python3.6', '/home/Peter.Kerins/anaconda3/envs/geoml/lib/python3.6/lib-dynload', '', '/home/Peter.Kerins/anaconda3/envs/geoml/lib/python3.6/site-packages', '/home/Peter.Kerins/anaconda3/envs/geoml/lib/python3.6/site-packages/IPython/extensions', '/home/Peter.Kerins/.ipython', '/home/Peter.Kerins/UrbanLandUse']


## Preparation

### Set key variables

In [2]:
# core
data_root='/data/phase_iv/'

resolution = 5  # Lx:15 S2:10

# tiling
tile_resolution = resolution
tile_size = 256
tile_pad = 32

look_window = 17
batch_size = 128

# misc
s2_bands=['blue','green','red','nir','swir1','swir2','alpha']; suffix='BGRNS1S2A'  # S2, Lx

# ground truth source: aue, aue+osm, aue+osm2
label_suffix = 'aue'
label_lot = '0'
resolution = 5
resampling = 'bilinear'
processing = None
source = 's2'

In [3]:
category_label = {0:'Open Space',1:'Non-Residential',\
                   2:'Residential Atomistic',3:'Residential Informal Subdivision',\
                   4:'Residential Formal Subdivision',5:'Residential Housing Project',\
                   6:'Roads',7:'Study Area',8:'Labeled Study Area',254:'No Data',255:'No Label'}

cats_map = {}
cats_map[0] = 0
cats_map[1] = 1
cats_map[2] = 2
cats_map[3] = 2
cats_map[4] = 2
cats_map[5] = 2

### Set input stack and model parameters

In [4]:
window = 17

# bands stuff outdated! needs to be reconciled with catalog filtering
# will ignore for the moment since this is a bigger fix...
# haven't done any examples yet incorporating additional chips beyond s2
# into construction of a training sample
bands_vir=s2_bands[:-1]
bands_sar=None
bands_ndvi=None
bands_ndbi=None
bands_osm=None

# this can get updated when cloudmasking is added
haze_removal = False

batch_size = 128
balancing = None

# move as appropriate

model_id = '3cat_14ct_green_2017_2-img-bl'
unflatten_input = True # is the model a cnn?
n_cats = 3 # number of categories

water_overwrite = False
water_mask = False



In [5]:
stack_label, feature_count = util_workflow.build_stack_label(
        bands_vir=bands_vir,
        bands_sar=bands_sar,
        bands_ndvi=bands_ndvi,
        bands_ndbi=bands_ndbi,
        bands_osm=bands_osm,)
print(stack_label, feature_count)

vir 6


***

### Load model

In [6]:
# category_weights_filename = data_root+'models/'+model_id+'_category_weights.pkl'
# category_weights = pickle.load( open( category_weights_filename, "rb" ) )
# weights = list(zip(*category_weights.items())[1])

network_filename = data_root+'models/'+model_id+'.hd5'
network = tf.keras.models.load_model(
    network_filename,
    custom_objects={'loss': 'categorical_crossentropy'},
    compile=True
)
# network = K.load_model(network_filename, custom_objects={'loss': 'categorical-crossentropy'})
network.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 17, 17, 6)    0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 17, 17, 32)   1760        input_2[0][0]                    
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 17, 17, 32)   0           conv2d_4[0][0]                   
________________________________________________________________________

***

## Apply model: score results
Apply model to some set of chips and compare its predictions to the actual LULC values

### Identify desired images

In [7]:
place_images = {}
place_images['hindupur']=['U']
# place_images['hindupur']=['U', 'V', 'W', 'X', 'Y', 'Z']
# place_images['singrauli']=['O','P','Q','R','S','T','U']
# place_images['vijayawada']=['H','I']
# place_images['jaipur']=['T','U','W','X','Y','Z']
# place_images['hyderabad']=['P','Q','R','S','T','U']
# place_images['sitapur']=['Q','R','T','U','V']
# place_images['kanpur']=['AH', 'AK', 'AL', 'AM', 'AN']
# place_images['belgaum']=['P','Q','R','S','T']
# place_images['parbhani']=['T','V','W','X','Y','Z']
# place_images['pune']=['P', 'Q', 'T', 'U', 'S']
# place_images['ahmedabad']= ['Z', 'V', 'W', 'X', 'Y', 'AA']
# place_images['malegaon']=  ['V', 'W', 'X', 'Y', 'Z']
# place_images['kolkata'] =  ['M','N','O','P','Q','R']
# place_images['mumbai']=['P','Q','R','S','U','V']

### Filter catalog to selected chips

In [8]:
df = util_chips.load_catalog()
print(len(df.index))

39281620


In [9]:
mask = pd.Series(data=np.zeros(len(df.index),dtype='uint8'), index=range(len(df)), dtype='uint8')

for place,image_list in place_images.items():
    for image in image_list:
        mask |= (df['city']==place) & (df['image']==image)

# straight away remove road samples
mask &= (df['lulc']!=6)

# filter others according to specifications
mask &= (df['gt_type']==label_suffix)
mask &= (df['gt_lot']==int(label_lot))
mask &= (df['source']==source)
mask &= (df['resolution']==int(resolution))
mask &= (df['resampling']==resampling)
mask &= (df['processing']==str(processing).lower())

print(np.sum(mask))

107891


In [10]:
#here for example we will just exclude all roads samples
df = df[mask]
df.reset_index(drop=True,inplace=True)
len(df)


107891

### Separate training and validation locales (if desired)

In [11]:
place_locales_paths = [
                       '/data/phase_iv/models/3cat_Hin_U-Z_place_locales.pkl'       ,
                       ]
# place_locales_paths = ['/data/phase_iv/models/3cat_Ahm_V-AA_place_locales.pkl',
#                        '/data/phase_iv/models/3cat_Bel_P-T_place_locales.pkl'       ,
#                        '/data/phase_iv/models/3cat_Hin_U-Z_place_locales.pkl'       ,
#                        '/data/phase_iv/models/3cat_Hyd_P-U_place_locales.pkl'       ,
#                        '/data/phase_iv/models/3cat_Jai_T-U+W-Z_place_locales.pkl'   ,
#                        '/data/phase_iv/models/3cat_Kan_AH+AK-AN_place_locales.pkl'  ,
#                        '/data/phase_iv/models/3cat_Mal_V-Z_place_locales.pkl'       ,
#                        '/data/phase_iv/models/3cat_Par_T+V-Z_place_locales.pkl',
#                        '/data/phase_iv/models/3cat_Pun_P-Q+S-U_place_locales.pkl',
#                        '/data/phase_iv/models/3cat_Sin_O-U_place_locales.pkl',
#                        '/data/phase_iv/models/3cat_Sit_Q-R+T-V_place_locales.pkl',
#                        '/data/phase_iv/models/3cat_Vij_H-I_place_locales.pkl',
#                        '/data/phase_iv/models/3cat_Kol_M-R_place_locales.pkl',
#                        '/data/phase_iv/models/3cat_Mum_P-V_place_locales.pkl'
#                        ]

In [12]:
combined_place_locales = {}
for place_locales_filename in place_locales_paths:
    with open(place_locales_filename, "rb") as f:
        place_locales = pickle.load(f,encoding='latin1')
    combined_place_locales.update(place_locales)
print(combined_place_locales)

{'hindupur': (array([ 0, 29, 28, 18, 17, 16, 15, 27,  9, 13, 20, 12, 24, 25,  3,  1,  5,
       11,  4, 26,  2]), array([ 6, 10,  7, 30,  8, 21, 22, 14, 19, 23]))}


In [13]:
df_t, df_v = util_chips.mask_locales(df, combined_place_locales)
print(len(df_t), len(df_v))

73026 34865


### Generate predictions on selected samples
Must be manually adjusted according to whether samples from training & validation locales will be scored independently

In [14]:
generator = CatalogGenerator(df,remapping='3cat',look_window=window,batch_size=batch_size,one_hot=3)
generator.reset()

#predict_generator(generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
predictions = network.predict_generator(generator, steps=generator.steps, verbose=1,
                                        use_multiprocessing=True, max_queue_size=40, workers=64,)

print(predictions.shape)

843/843 [==============================] - 19s 22ms/step
(107891, 3)


In [15]:
Yhat = predictions.argmax(axis=-1)
Y = generator.get_label_series().values
        
print("evaluate validation")
# hardcoded categories
categories=[0,1,2]
confusion = util_scoring.calc_confusion(Yhat,Y,categories)
recalls, precisions, accuracy = util_scoring.calc_confusion_details(confusion)

# Calculate f-score
beta = 2
f_score = (beta**2 + 1) * precisions * recalls / ( (beta**2 * precisions) + recalls )
f_score_open = f_score[0] 
f_score_nonres = f_score[1]  
f_score_res = f_score[2]  
f_score_roads = None#f_score[3]  
f_score_average = np.mean(f_score)

# expanding lists to match expected model_record stuff
recalls_expanded = [recalls[0],recalls[1],recalls[2],None]
precisions_expanded = [precisions[0],precisions[1],precisions[2],None]

# util_scoring.record_model_application(
#     model_id, notes, place + '(' + image + ')', label_suffix, resolution, stack_label, feature_count, 
#     generator.look_window,cats_map, 
#     confusion, recalls[0], recalls[1], recalls[2], recalls[3], precisions[0], precisions[1], precisions[2], precisions[3], accuracy, 
#     f_score_open, f_score_nonres, f_score_res, f_score_roads, f_score_average)

(107891,)
(107891,)
evaluate validation
0 55080
1 11915
2 40896
[[50964  1455  2661]
 [ 4426  5963  1526]
 [ 8574  1926 30396]]
107891 87323 0.809363153553123
